In [1]:
!pip install pandas spacy tqdm
!python -m spacy download ru_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 MB 2.9 MB/s eta 0:00:0000:0100:01m
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')


In [1]:
import pandas as pd
import os
import glob
import spacy
import json
from tqdm import tqdm

In [3]:
# 1. Объединение CSV файлов из директории GenCorpus
directory = 'GenCorpus'
output_file = os.path.join(directory, 'combined_data.csv')

In [5]:
all_data = []
csv_files = glob.glob(os.path.join(directory, '*.csv'))
print(f"Найдено {len(csv_files)} CSV файлов")

Найдено 10 CSV файлов


In [7]:
for file in csv_files:
    df = pd.read_csv(file)
    all_data.append(df)
    print(f"Добавлен файл: {file}, форма: {df.shape}")

combined_df = pd.concat(all_data, ignore_index=True)
combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)
combined_df.to_csv(output_file, index=False)
print(f"Данные объединены и сохранены в {output_file}, итоговая форма: {combined_df.shape}")

Добавлен файл: GenCorpus/gpt_3_paraphrased.csv, форма: (1096, 2)
Добавлен файл: GenCorpus/claude_2_paraphrased.csv, форма: (1096, 2)
Добавлен файл: GenCorpus/gpt_2_paraphrased.csv, форма: (1095, 2)
Добавлен файл: GenCorpus/deepseek_1_paraphrased.csv, форма: (1096, 2)
Добавлен файл: GenCorpus/deepseek_paraphrased.csv, форма: (1096, 2)
Добавлен файл: GenCorpus/gemini_1_paraphrased.csv, форма: (1085, 2)
Добавлен файл: GenCorpus/gpt_1_paraphrased.csv, форма: (1096, 2)
Добавлен файл: GenCorpus/deepseek_2_paraphrased.csv, форма: (1102, 2)
Добавлен файл: GenCorpus/llama_paraphrased.csv, форма: (1077, 2)
Добавлен файл: GenCorpus/claude_1_paraphrased.csv, форма: (1096, 2)
Данные объединены и сохранены в GenCorpus/combined_data.csv, итоговая форма: (10935, 2)


In [11]:
# 2. Аннотация второй колонки с помощью spaCy
nlp = spacy.load("ru_core_news_md")

# Создаем список для хранения аннотаций в JSON-совместимом формате
annotations_data = []

In [13]:
print("Выполняем аннотацию с помощью spaCy...")
for i, text in enumerate(tqdm(combined_df['paraphrased_text'])):
    doc = nlp(text)
    
    # Преобразуем данные spaCy в JSON-совместимый формат
    tokens = [{"text": token.text, 
               "lemma": token.lemma_, 
               "pos": token.pos_, 
               "tag": token.tag_,
               "dep": token.dep_,
               "is_stop": token.is_stop} for token in doc]
    
    entities = [{"text": ent.text, 
                 "start": ent.start_char, 
                 "end": ent.end_char, 
                 "label": ent.label_} for ent in doc.ents]
    
    annotations_data.append({
        "original_text": combined_df['original_text'][i],
        "paraphrased_text": text,
        "tokens": tokens,
        "entities": entities
    })

Выполняем аннотацию с помощью spaCy...


100%|█████████████████████████████████████| 10935/10935 [06:56<00:00, 26.25it/s]


In [15]:
# Сохраняем аннотированные данные в JSON
annotated_file = os.path.join(directory, 'annotated_data.json')
with open(annotated_file, 'w', encoding='utf-8') as f:
    json.dump(annotations_data, f, ensure_ascii=False, indent=2)

print(f"Аннотированные данные сохранены в {annotated_file}")

Аннотированные данные сохранены в GenCorpus/annotated_data.json
